<a href="https://colab.research.google.com/github/vviedienieiev/Projects/blob/master/Ukrainian_language_share/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install polyglot  
!pip install pyicu     
!pip install Morfessor   
!pip install pycld2 
!pip install langdetect

In [39]:
from polyglot.detect import Detector
from polyglot.detect.base import logger
logger.disabled = True
from langdetect import detect
import numpy as np
import string
from collections import Counter
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
data = np.load("all_message.npy",allow_pickle=True) #get the data

In [5]:
exclude_punctuation = set(string.punctuation) #list of all punctuation symbols
exclude_numbers = string.digits #list of all digits
exclude_english = string.ascii_letters #list of all english upper and lower case letters
exclude_other = ['—','́','«', '»','🌚', '🤔','…', '😆','🤡','–', '🤗','\u200b', '😅', '🙏', '🏻', '😘', '🥺', '😌',
                 '😬','😢', '🤦', '♂', '️', '👍', '😏', '😨', '😂', '😃', '😐',
                 '🤷','♀', '🔺', '😭', '😁', '◾', '🙈', '🥳', '±', '👴', '😈', '😶', '\u200d'] # list of other symbols

In [6]:
final_set = [text for text in data if text is not None] #remove empty elements
final_set = " ".join(final_set) #convert to string
final_set = ''.join(ch for ch in final_set if ch not in exclude_punctuation) #remove punctuation
final_set = ''.join(ch for ch in final_set if ch not in exclude_numbers) # remove numbers
final_set = ''.join(ch for ch in final_set if ch not in exclude_english) # remove english characters
final_set = ''.join(ch for ch in final_set if ch not in exclude_other) # remove other symbols
final_set = final_set.split(" ") # make tokenezation
final_set = [word for word in final_set if len(word) > 2] # remove all small words


In [35]:
##### MODEL 1 #####
languages = [Detector(word, quiet=True).language.name for word in final_set] # detect language of the word
Counter(languages) # count amount of words within each language


Counter({'Abkhazian': 1109,
         'Bashkir': 701,
         'Belarusian': 12838,
         'Bulgarian': 12048,
         'Kazakh': 1720,
         'Kyrgyz': 2673,
         'Macedonian': 5911,
         'Mongolian': 1259,
         'Romanian': 2724,
         'Russian': 164682,
         'Serbian': 29297,
         'Tajik': 2721,
         'Tatar': 1287,
         'Turkmen': 1304,
         'Ukrainian': 272056,
         'Uyghur': 1240,
         'Uzbek': 2775,
         'un': 13972})

In [42]:
#Model 2
detectors = [Detector(word, quiet=True) for word in final_set]
languages = [detector.language.name if detector.reliable else 'not_detected' for detector in detectors]
Counter(languages)

Counter({'Abkhazian': 100,
         'Belarusian': 2449,
         'Bulgarian': 310,
         'Kyrgyz': 82,
         'Macedonian': 344,
         'Romanian': 300,
         'Russian': 16667,
         'Serbian': 900,
         'Tajik': 200,
         'Ukrainian': 83115,
         'Uyghur': 202,
         'Uzbek': 200,
         'not_detected': 425448})

In [20]:
##### MODEL 3 #####
#Ru share
def get_ru_words():
  url = "http://slovardalja.net/"
  response = requests.get(url)
  response.encoding = 'cp1251'
  soup = BeautifulSoup(response.text, "html.parser")
  links = soup.select(".chara")
  chars = [link.text.lower() for link in links]
  links = ["http://slovardalja.net/"+link["href"] for link in links]
  all_words = []
  for link in links:
      response = requests.get(link)
      response.encoding = 'cp1251'
      soup = BeautifulSoup(response.text, "html.parser")
      words = soup.select(".chara strong")
      all_words.extend(words)
  all_words = [word.text.lower() for word in all_words if word.text.lower() not in chars]
  all_words_str = "".join(all_words)
  all_words_final = "".join(ch for ch in all_words_str if ch in chars)
  return all_words_final

#UA Share  
def get_ua_words():
  url = "http://ukrlit.org/slovnyk/slovnyk_ukrainskoi_movy_v_11_tomakh"
  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser")
  chars = soup.select(".letter p")
  chars = [char.text for char in chars]
  links = soup.select(".letters__dropdown_opacity a")
  links = ["http://ukrlit.org"+link["href"] for link in links]
  all_words = []
  for link in links:
      response = requests.get(link)
      soup = BeautifulSoup(response.text, "html.parser")
      words = soup.select(".word-list__item a")
      words = [word.text.lower() for word in words]
      all_words.extend(words)
  all_words_str = "".join(all_words)
  all_words_final = "".join(ch for ch in all_words_str if ch in chars)
  return all_words_final


#Calculate special chars in initial dataset
final_set_model_2 = " ".join(final_set)
ukr = final_set_model_2.count("і")+final_set_model_2.count("є")+final_set_model_2.count("ї")
rus = final_set_model_2.count("ы")+final_set_model_2.count("ъ")+final_set_model_2.count("э")

all_words_ru = get_ru_words()
share_ru = (all_words_ru.count("ы")+all_words_ru.count("ъ")+all_words_ru.count("э"))/len(all_words_ru)

all_words_ua = get_ua_words()
share_ua = (all_words_ua.count("і")+all_words_ua.count("є")+all_words_ua.count("ї"))/len(all_words_ua)

175089 11251 0.06037887732102608


In [23]:
print(f"Amount of ukrainian chars in dataset {ukr}")
print(f"Amount of russian chars in dataset {rus}")
print(f"Share of russian chars in dataset {rus/(rus+ukr)}")
print(f"Frequence of ukrainian chars among ukrainian words {share_ua*100}%")
print(f"Frequence of russian chars among ukrainian words {share_ru*100}%")
print(f"Adjusted amount of russian words {rus/(rus+ukr)*(share_ua/share_ru)*100}%")



Amount of ukrainian chars in dataset 175089
Amount of russian chars in dataset 11251
Share of russian chars in dataset 0.06037887732102608
Frequence of ukrainian chars among ukrainian words 4.505183548063595%
Frequence of russian chars among ukrainian words 2.264657623863714%
Adjusted amount of russian words 12%
